In [ ]:
# Downloading Dataset
!wget https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/movielens_data/ratings.csv

In [ ]:
# Downloading the Movies Dataset
!wget https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/movielens_data/movies.csv

In [ ]:
#instalação do scikit-surprise Package
!pip install scikit-surprise

In [ ]:
#importação das bibliotecas básicas
import numpy as np
import pandas as pd

In [ ]:
#Carregamento dos dados dos arquivos
movies = pd.read_csv('movies.csv')

movies.shape

In [ ]:
#Carregamento dos dados dos arquivos
ratings = pd.read_csv('ratings.csv')

ratings.shape

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
#Remoção da coluna timestamp
ratings = ratings.drop(['timestamp'], axis = 1)

In [ ]:
#criação de um novo arquivo CSV para os dados de avaliações devido ao fato de que a biblioteca Surprise apenas aceitará arquivo no formato CSV como Entrada
#Não especificaremos cabeçalho (None)
#Não utilizaremos indices
ratings.to_csv("ratings_modified.csv", header = None, index = False)

In [ ]:
#Após as modificações, verifiquemos o formato carregado dos dados
x = pd.read_csv('ratings_modified.csv')
x.head()

In [ ]:
#Vamos importar o dataset para o leitor do Surprise
from surprise import Dataset, Reader

#Primeiramente, precisamos especirficar o caminho do arquivo e os parâmetros de letura requeriso para o carregamento dos dados
file_path = 'ratings_modified.csv'
reader = Reader(line_format='user item rating', sep=',', rating_scale = (1,5))

#Carregamento do Dataset:
data = Dataset.load_from_file(file_path, reader=reader)

In [ ]:
#Vamos construir um dataset de treinamento
train = data.build_full_trainset()

#Numero de usuários e itens
print('Número de usuários na base de dados:', train.n_users)
print('Número de itens na base de dados:', train.n_items)

In [ ]:
#Construção do modelo

from surprise import KNNWithMeans

#Filtragem colaborativa baseada no usuário:
my_sim_option = {'name':'pearson', 'user_based':True}

#Modelo KNN como backend
algo = KNNWithMeans(k = 15, min_k = 5, 
    sim_options = my_sim_option, verbose = True
    )

In [ ]:
#Verifiquemos a acurácia do modelo
from surprise.model_selection import  cross_validate

#Validação cruzada
results = cross_validate(algo = algo, 
                         data = data, 
                         measures=['RMSE'], 
                         cv = 5, 
                         return_train_measures=True)
print(results['test_rmse'].mean())


In [ ]:
#Treinamento do modelo em nosso Dataset
algo.fit(train)

In [ ]:
#Criação de um dicionário para mapear o ID do Filme (MovieID) e os títulos dos filmes
movie_id_to_title_map = {}

for m_id , title in zip(movies['movieId'].values , movies['title'].values):
    movie_id_to_title_map[str(m_id)] = title

#Verificação do dicionário criado:
movie_id_to_title_map

In [ ]:
#Predição em tempo real

#O quanto o usuário de user id 5 avaliaria o item de ID 105?
val = algo.predict(uid = '1', iid = '31')
print(movie_id_to_title_map[val[1]], val[3])

In [ ]:
#Melhores previsões
from collections import defaultdict

def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
#Construindo um conjunto de testes interativo, pois previsões sobre os dados de treinamento é um erro
testdata = train.build_anti_testset()
predictions = algo.test(testdata)
top_n = get_top_n(predictions, n = 10)

In [ ]:
#Vamos criar uma função de busca de todos os filmes assistidos pelos usuários:
def PreviousMoviedUserWatched(user_df , user_id , item_map):
    user_df = user_df[user_df.iloc[: , 0] == user_id]
    for movie , rating in zip(user_df.iloc[:,1].values , user_df.iloc[:,2].values):
        print(item_map[str(movie)] , rating)

In [ ]:
#Vamos criar uma função de previsão de filmes para usuários baseandos nos filmes assistidos anteriormente:
def UserPredictions(user_id , top_n , item_map):
  print("Predictions for User Id : " , user_id)
  user_ratings = top_n[user_id]
  for item_id , rating in user_ratings :
    print(item_map[item_id] , " : " , rating)

In [ ]:
PreviousMovieUserWatched(ratings, 1, movie_id_to_title_map)

In [ ]:
UserPredictions('50' , top_n , movie_id_to_title_map)